In [1]:
import datetime as dt
import pandas as pd
import pandas_datareader as pdr
import matplotlib.pyplot as plt
import xlwings as xl

In [2]:
tDeltas = {'1Y': dt.timedelta(days=365),
           '2Y': dt.timedelta(days=365*2),
           '3Y': dt.timedelta(days=365*3),
           '4Y': dt.timedelta(days=365*4),
           '5Y': dt.timedelta(weeks=260.714),
           '10Y': dt.timedelta(weeks=521.429)}

# Setting Dates_________#
now = dt.date.today()
start = now - tDeltas['1Y']

security_name = 'es=f'.upper()

In [6]:
# price_data = pdr.DataReader(security_name, data_source='yahoo', start=start, end=now)
price_data = pd.read_csv('Sample Datasets/AEX D1 10Y.csv', parse_dates=['Date'], index_col=['Date'])
price_data.drop(columns=['Vol.', 'Change %'], inplace=True)
price_data.sort_index(inplace=True)
price_data

,Price,Open,High,Low
Date,,,,
2010-06-23,335.11,336.05,339.07,333.72
2010-06-24,329.22,337.39,337.42,328.49
2010-06-25,325.99,329.67,330.34,325.26
2010-06-28,330.46,327.33,331.04,325.39
2010-06-29,319.03,326.35,326.50,318.14
...,...,...,...,...
2020-07-17,573.80,575.45,575.97,572.27
2020-07-20,579.14,573.48,580.46,569.46
2020-07-21,581.29,584.04,587.03,580.42


In [7]:
# creating open 2open & high 2 low returns column
price_data['O2O %'] = (price_data['Open'].pct_change() * 100).round(3)
price_data['H2L %'] = (((price_data['High'] - price_data['Low'])/price_data['Low']) * 100).round(3)
price_data.sort_index(ascending=False, inplace=True)
price_data

,Price,Open,High,Low,O2O %,H2L %
Date,,,,,,
2020-07-23,576.67,572.97,580.74,572.97,-0.939,1.356
2020-07-22,572.90,578.40,579.21,571.35,-0.966,1.376
2020-07-21,581.29,584.04,587.03,580.42,1.841,1.139
2020-07-20,579.14,573.48,580.46,569.46,-0.342,1.932
2020-07-17,573.80,575.45,575.97,572.27,0.277,0.647
...,...,...,...,...,...,...
2010-06-29,319.03,326.35,326.50,318.14,-0.299,2.628
2010-06-28,330.46,327.33,331.04,325.39,-0.710,1.736
2010-06-25,325.99,329.67,330.34,325.26,-2.288,1.562


In [60]:
# Lowest & highest open 2 open changes
min_ = price_data['O2O %'].min()
max_ = price_data['O2O %'].max()

In [61]:
largest_5th = price_data['O2O %'].nlargest(10).iloc[-1]
smallest_5th = price_data['O2O %'].nsmallest(10).iloc[-1]
largest_5th

4.294

In [62]:
smallest_5th

-5.006

In [63]:
# Divisor
k = int(abs((smallest_5th - largest_5th)/0.3))
k

31

In [75]:
#________________Distribution Functions_________________#
def frequency(array, dataframe=None, col_name=None):
    """Returns the frequency of values within
    a start & end range, in a given DataFrames column"""

    start = array[0]
    end = array[1]
    
    k = dataframe[dataframe[col_name].between(start, end, inclusive=True)]
    return k[col_name].count()

In [65]:
# creating bin series
bin_ = [min_ - 0.3, smallest_5th, ]
for i in range(k + 1):
    smallest_5th += 0.3
    bin_.append(smallest_5th)
bin_.append(max_ + 0.3)
bin_Series = pd.Series(bin_).round(1)
bin_Series

0    -9.8
1    -5.0
2    -4.7
3    -4.4
4    -4.1
5    -3.8
6    -3.5
7    -3.2
8    -2.9
9    -2.6
10   -2.3
11   -2.0
12   -1.7
13   -1.4
14   -1.1
15   -0.8
16   -0.5
17   -0.2
18    0.1
19    0.4
20    0.7
21    1.0
22    1.3
23    1.6
24    1.9
25    2.2
26    2.5
27    2.8
28    3.1
29    3.4
30    3.7
31    4.0
32    4.3
33    4.6
34    9.3
dtype: float64

In [76]:
frequency_series = bin_Series.rolling(2).apply(frequency, raw=True, kwargs={'dataframe': price_data, 'col_name': 'O2O %'})
frequency_series.fillna(0, inplace=True)
frequency_series = frequency_series.astype('int32', copy=False)
frequency_series

0       0
1      10
2       1
3       0
4       1
5       2
6       5
7       5
8       9
9      11
10     20
11     38
12     39
13     60
14    104
15    141
16    222
17    285
18    374
19    395
20    322
21    200
22    130
23     76
24     47
25     26
26     23
27     14
28      7
29      8
30      5
31      1
32      1
33      2
34      7
dtype: int32

In [77]:
frequency_table = pd.concat([bin_Series, frequency_series], axis=1)         # TODO merge Frequency Table
frequency_table.columns = ['bin', 'Frequency']
frequency_table

,bin,Frequency
0,-9.8,0
1,-5.0,10
2,-4.7,1
3,-4.4,0
4,-4.1,1
5,-3.8,2
6,-3.5,5
7,-3.2,5
8,-2.9,9
9,-2.6,11


In [70]:
len(price_data['O2O %'])

2582

In [78]:
# Probability & Cu-probability
prob_Series = ((frequency_table['Frequency']/2581) * 100).round(2)
cumprob = (((frequency_table['Frequency']/2581) * 100).round(2)).cumsum()

prob_table = pd.concat([prob_Series, cumprob], axis=1)               # Probability table
prob_table.columns = ['Probability %', 'Cum Probability %']

probability_distribution = pd.concat([frequency_table, prob_table], axis=1)
probability_distribution

,bin,Frequency,Probability %,Cum Probability %
0,-9.8,0,0.00,0.00
1,-5.0,10,0.39,0.39
2,-4.7,1,0.04,0.43
3,-4.4,0,0.00,0.43
4,-4.1,1,0.04,0.47
5,-3.8,2,0.08,0.55
6,-3.5,5,0.19,0.74
7,-3.2,5,0.19,0.93
8,-2.9,9,0.35,1.28
9,-2.6,11,0.43,1.71


In [14]:
# lowest & highest low to high returns
h2lmin = 0
h2lmax = price_data['H2L %'].max()
h2lmax

9.752

In [15]:
# 5th largest & smallest h2l returns
h2l_5th_largest = price_data['H2L %'].nlargest(5).iloc[-1]
h2l_5th_largest

7.324

In [17]:
# Divisor
k2 = int(abs(h2l_5th_largest/0.3))
k2

24

In [18]:
# creating bin series
bin2 = [0, ]
for i in range(k2 + 1):
    h2lmin += 0.3
    bin2.append(h2lmin)
bin2.append(h2lmax)

h2l_bin_Series = pd.Series(bin2).round(3)
h2l_bin_Series

NameError: name 'k2' is not defined

In [17]:
h2l_frequency_series = h2l_bin_Series.rolling(2).apply(frequency, raw=True, kwargs={'dataframe': price_data, 'col_name': 'H2L %'})
h2l_frequency_table = pd.concat([h2l_bin_Series, h2l_frequency_series], axis=1)         # TODO merge Frequency Table
h2l_frequency_table.columns = ['bin', 'Frequency']
h2l_frequency_table.fillna(value=0, inplace=True)

h2l_frequency_table

NameError: name 'h2l_bin_Series' is not defined

In [16]:
# Probability & Cu-probability
h2l_prob_Series = (h2l_frequency_table['Frequency']/len(price_data['H2L %']) * 100).round(2)
h2l_cumprob = ((h2l_frequency_table['Frequency']/len(price_data['H2L %']) * 100).round(2)).cumsum()

h2l_prob_table = pd.concat([h2l_prob_Series, h2l_cumprob], axis=1)               # Probability table
h2l_prob_table.columns = ['Probability %', 'Cum Probability %']

h2l_probability_distribution = pd.concat([h2l_frequency_table, h2l_prob_table], axis=1)      # TODO merge probability distribution

h2l_probability_distribution

NameError: name 'h2l_frequency_table' is not defined